In [1]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [49]:
def getInferenceResults(filename):
    with open (filename, 'r') as f:
        lines = f.readlines()
        results_dict = dict()
        for line in lines:

            probability = float(line.split(' ')[1])
            start = 0
            while(line[start] != '('):
                start = start + 1
            start = start + 1
            end = start
            while(line[end]!=')'):
                end = end + 1

            content = line[start:end].split(',')
#             print(content,probability)
            if line[0] == 'E':
                if content[0]+'-'+content[1] in results_dict and results_dict[content[0]+'-'+content[1]][1] < probability:
                    results_dict[content[0]+'-'+content[1]] = (content[2], probability)
                elif content[0]+'-'+content[1] not in results_dict:
                    results_dict[content[0]+'-'+content[1]] = (content[2], probability)

            else:
                if content[0]+'-'+content[1]+'-'+content[2] in results_dict and results_dict[content[0]+'-'+content[1]+'-'+content[2]][1] < probability:
                    results_dict[content[0]+'-'+content[1]+'-'+content[2]] = (content[3],probability)
                elif content[0]+'-'+content[1]+'-'+content[2] not in results_dict:
                    results_dict[content[0]+'-'+content[1]+'-'+content[2]] = (content[3],probability)
#         print(results_dict)
        return results_dict

def getClassifierReport(file,label,labels):
    data = pd.read_csv(file) 

    predicted_tags = data['Predicted']
    gold_truths = data['Gold Truth']
    print('The Classification Report of ',label)
    print(classification_report(predicted_tags,gold_truths))

def getAlchemyReport(file,label,mln_results,labels):
    THRESHOLD = 0.1
    data = pd.read_csv(file)
    actual_tags = []
    predicted_tags = []
    
#     print(mln_results)
    if label == 'Base Classifier':
        THRESHOLD = 0.1
        for sentenceID,tokenID, gold_truth in zip(data['SentenceID'],data['tokenID'],data['Gold Truth']):
            key = str(sentenceID) + '-'+str(tokenID)
            if 'Other' not in gold_truth:
                if len(gold_truth) > 2:
                    actual_tags.append(gold_truth[2:])
                else:
                    actual_tags.append(gold_truth)
                if key in mln_results.keys() and mln_results[key][1] > THRESHOLD:
                    predicted_tags.append(mln_results[key][0])
                else:
                    predicted_tags.append('O')

        
    else:
        THRESHOLD = 0.005
        for sentenceID,token1,token2,gold_truth in zip(data['SentenceID'],data['token1'],data['token2'],data['Gold Truth']):
            key = str(sentenceID) + '-'+str(token1)+'-'+str(token2)
            
#             if gold_truth != 'None':
            actual_tags.append(gold_truth)
            if key in mln_results.keys() and mln_results[key][1] > THRESHOLD:
                predicted_tags.append(mln_results[key][0])
            else:
                    predicted_tags.append('None')
        
#     print("Predicted Tag----------Actual Tag")
#     for i in range(len(actual_tags)):
#         print(predicted_tags[i]+'------------'+actual_tags[i])
    
#     print('The Classification Report(Alchemy) of ',label)
    

    print(classification_report(predicted_tags,actual_tags))
    print(labels)
    print(confusion_matrix(actual_tags,predicted_tags,labels=labels))
    
if __name__=='__main__':
    mln_results = getInferenceResults('inference_results_10.results')
    
#     print(mln_results['1-21-5'][1]*100)

#     print(mln_results)

    entities = ['Peop','Org','O','Loc']
    relations = ['Work_For','Live_In','Kill','Located_In','None','OrgBased_In']
    getClassifierReport("relation_classifier.csv",'Relation Classifier',relations)
    getAlchemyReport('relation_classifier_10.csv','Relation Classifier',mln_results,relations)
    
    getClassifierReport("base_classifier_10.csv",'Base Classifier',entities)
    getAlchemyReport('base_classifier_10.csv','Base Classifier',mln_results,entities)
    
    
    
    
    
    

The Classification Report of  Relation Classifier
              precision    recall  f1-score   support

        Kill       0.99      0.58      0.73       336
     Live_In       0.85      0.65      0.73       476
  Located_In       0.95      0.49      0.65       530
        None       0.85      0.98      0.91      4904
 OrgBased_In       0.87      0.63      0.73       435
    Work_For       0.88      0.65      0.75       382

    accuracy                           0.86      7063
   macro avg       0.90      0.66      0.75      7063
weighted avg       0.87      0.86      0.85      7063

              precision    recall  f1-score   support

        Kill       0.50      0.50      0.50         2
     Live_In       0.50      1.00      0.67         2
  Located_In       0.50      1.00      0.67         1
        None       0.00      0.00      0.00         7
 OrgBased_In       0.56      0.83      0.67         6
    Work_For       0.67      1.00      0.80         2

    accuracy               

In [22]:

    
    
if __name__=='__main__':
    getReport("base_classifier_10.csv",'Base Classifier')
#     getReport("relation_classifier.csv",'Relation Classifier')

The Classification Report of  Base Classifier
              precision    recall  f1-score   support

       B-Loc       0.88      0.88      0.88        16
       B-Org       0.73      0.89      0.80         9
     B-Other       1.00      0.75      0.86         4
      B-Peop       0.89      0.57      0.70        14
       I-Loc       1.00      0.50      0.67         6
       I-Org       0.61      0.92      0.73        12
     I-Other       1.00      0.80      0.89         5
      I-Peop       0.92      1.00      0.96        11
           O       0.97      0.98      0.98       232

    accuracy                           0.94       309
   macro avg       0.89      0.81      0.83       309
weighted avg       0.94      0.94      0.93       309

